In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import time
from datetime import datetime
from urllib.error import HTTPError
import requests

In [2]:
df = pd.read_csv("links.csv", index_col=[0])
df = df.drop_duplicates( keep='last')
df = df.reset_index(drop=True)

In [3]:
df['location'] = df['link'].str.split('/').str[1]
df['type'] = df['link'].str.split('/').str[3]

In [4]:
hdr = {'User-Agent': 'Mozilla/5.0'}
price_df = []
size_df = []
rooms_df = []
num = 1
for i in df['link']:
    site = "https://ingatlan.com" + str(i)
    
    try:
        response = requests.get(site, headers=hdr)
        response.raise_for_status()
    except:
        print("failure")
        price_df.append("NA")
        rooms_df.append("NA")
        size_df.append("NA")
     
    else:
        site= "https://ingatlan.com" + str(i)
        req = Request(site,headers=hdr)
        page = urlopen(req)
        soup = BeautifulSoup(page)
        price = soup.find_all('span', {'fw-bold fs-5 text-nowrap'})
        price = price[0].get_text()
        price = price.strip()
        price_df.append(price)

        rooms = soup.find_all('span', {'fw-bold fs-5'})
        rooms = rooms[1].get_text()
        rooms = rooms.strip()
        rooms_df.append(rooms)

        size = soup.find_all('span', {'fw-bold fs-5'})
        size = size[0].get_text()
        size = size.strip()
        size_df.append(size)
        
        print('done: ' + str(num) + " from " + str(len(df)))
        num = num + 1
        #time.sleep(0.5) #dont want to overwhelm the site

done: 1 from 51150
done: 2 from 51150
done: 3 from 51150
done: 4 from 51150
done: 5 from 51150
done: 6 from 51150
done: 7 from 51150
done: 8 from 51150
done: 9 from 51150
done: 10 from 51150
done: 11 from 51150
done: 12 from 51150
done: 13 from 51150
done: 14 from 51150
done: 15 from 51150
done: 16 from 51150
done: 17 from 51150
done: 18 from 51150
done: 19 from 51150
done: 20 from 51150
done: 21 from 51150
done: 22 from 51150
done: 23 from 51150
done: 24 from 51150
done: 25 from 51150
done: 26 from 51150
done: 27 from 51150
done: 28 from 51150
done: 29 from 51150
done: 30 from 51150
done: 31 from 51150
done: 32 from 51150
done: 33 from 51150
done: 34 from 51150
done: 35 from 51150
done: 36 from 51150
done: 37 from 51150
done: 38 from 51150
done: 39 from 51150
done: 40 from 51150
done: 41 from 51150
done: 42 from 51150
done: 43 from 51150
done: 44 from 51150
done: 45 from 51150
done: 46 from 51150
done: 47 from 51150
done: 48 from 51150
done: 49 from 51150
done: 50 from 51150
done: 51 

In [ ]:
df['size'] = size_df
df['rooms'] = rooms_df
df['price'] = price_df

In [ ]:

now = datetime.now() # current date and time
date = now.strftime("%Y-%m-%d")
df.to_csv("dataframe-" + date + ".csv", sep = ";")